In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
path = './main_data/'
path2 = './main_data/events/'

In [3]:
organizations = pd.read_csv(path2 + 'organizations_updated.csv', sep=';')

In [4]:
click = pd.read_csv(path + 'click.txt', sep=';', encoding= 'unicode_escape', error_bad_lines=False)

b'Skipping line 431497: expected 6 fields, saw 7\nSkipping line 431498: expected 6 fields, saw 7\n'
b'Skipping line 1071166: expected 6 fields, saw 7\nSkipping line 1071167: expected 6 fields, saw 7\n'
b'Skipping line 1422097: expected 6 fields, saw 7\n'
b'Skipping line 1537079: expected 6 fields, saw 7\n'


In [5]:
n_clicks_before = click.shape[0]
click = click.drop_duplicates(['create_time', 'user_id'])
n_clicks_after = click.shape[0]

print(f'Decreased # clicks from {n_clicks_before} to {n_clicks_after}')

Decreased # clicks from 1596723 to 1031660


In [6]:
click.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031660 entries, 0 to 1596722
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   create_time         1031660 non-null  object
 1   create_date         1031660 non-null  object
 2   user_phone_details  1031660 non-null  object
 3   buyer_mobile_phone  1031660 non-null  object
 4   user_id             1031660 non-null  object
 5   url                 1031660 non-null  object
dtypes: object(6)
memory usage: 55.1+ MB


In [7]:
events = pd.read_csv(path2 + 'events.csv', sep=';', error_bad_lines=False)

In [8]:
events = events.rename(columns= {'Ссылка на покупку билета': 'url',
                                 'ID': 'session_id',
                                 'Название события': 'session_name',
                                 'Дополнительная ссылка на покупку билета': 'additional_url',
                                 'Организатор мероприятия': 'organization_name'})

In [9]:
events['session_identity'] = events['session_name'].apply(lambda x: x.split()[0].strip())

In [10]:
events = events.loc[(events['url'].notna())|(events['additional_url'].notna())]

In [11]:
events.head()

,session_id,session_name,organization_name,url,additional_url,session_identity
0,36,Спектакль «Касатка»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль
1,40,Спектакль «Гамлет»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль
2,86,Спектакль «Горе от ума»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль
3,109,Спектакль «Женитьба»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль
4,139,Спектакль «Куклы»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль


In [12]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42151 entries, 0 to 42153
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   session_id         42151 non-null  int64 
 1   session_name       42151 non-null  object
 2   organization_name  42151 non-null  object
 3   url                42151 non-null  object
 4   additional_url     3817 non-null   object
 5   session_identity   42151 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.3+ MB


In [13]:
events_pushka_accepted_30122021 = pd.read_csv(path2 + 'events_pushka_accepted_30122021.csv', sep = ',')

In [14]:
events_pushka_accepted_30122021.head(50)

,entity._id,entity.name,entity.saleLink,entity.additionalSaleLinks.0,entity.organization._id,entity.organization.name
0,36,Спектакль «Касатка»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
1,86,Спектакль «Горе от ума»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
2,86,Спектакль «Горе от ума»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
3,139,Спектакль «Куклы»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
4,150,Спектакль «Филумена Мартурано»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
5,155,Спектакль в фойе «С любовью не шутят»,https://quicktickets.ru/belgorodskiy-dramatich...,https://quicktickets.ru/belgorodskiy-dramatich...,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
6,890,Спектакль «Терешечка»,https://quicktickets.ru/belgorodskiy-gosudarst...,NaN,10.0,ГБУК «Белгородский государственный театр кукол»
7,1591,Спектакль «История солдата»,https://quicktickets.ru/belgorodskiy-gosudarst...,NaN,10.0,ГБУК «Белгородский государственный театр кукол»
8,1878,"Спектакль «Примадонны, или Шоу продолжается»",https://quicktickets.ru/teatrkaluga/e10,NaN,214.0,ГБУК КО «Калужский ордена Трудового Красного З...
9,1878,"Спектакль «Примадонны, или Шоу продолжается»",https://quicktickets.ru/teatrkaluga/e10,NaN,214.0,ГБУК КО «Калужский ордена Трудового Красного З...


In [15]:
events_pushka_accepted_30122021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47856 entries, 0 to 47855
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   entity._id                    47856 non-null  int64  
 1   entity.name                   47220 non-null  object 
 2   entity.saleLink               47218 non-null  object 
 3   entity.additionalSaleLinks.0  4636 non-null   object 
 4   entity.organization._id       47220 non-null  float64
 5   entity.organization.name      47220 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.2+ MB


In [16]:
events_pushka_accepted_30122021 = events_pushka_accepted_30122021[(events_pushka_accepted_30122021['entity.saleLink'].notna())|
                                                                 (events_pushka_accepted_30122021['entity.additionalSaleLinks.0'])]

In [17]:
events_pushka_accepted_30122021['session_identity'] = events_pushka_accepted_30122021['entity.name'
                                                                                     ].apply(lambda x:
                                                                                            x.split()[0].strip())

In [18]:
events_pushka_accepted_30122021 = events_pushka_accepted_30122021.rename(columns= {'entity.saleLink': 'url',
                                                                                  'entity._id': 'session_id',
                                                                                  'entity.name': 'session_name',
                                                                                  'entity.additionalSaleLinks.0': 'additional_url',
                                                                                  'entity.organization._id': 'organization_id',
                                                                                  'entity.organization.name': 'organization_name'})

In [19]:
events_all = pd.concat([events_pushka_accepted_30122021, events])

In [20]:
events_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89369 entries, 0 to 42153
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   session_id         89369 non-null  int64  
 1   session_name       89369 non-null  object 
 2   url                89369 non-null  object 
 3   additional_url     8453 non-null   object 
 4   organization_id    47218 non-null  float64
 5   organization_name  89369 non-null  object 
 6   session_identity   89369 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 5.5+ MB


In [21]:
events_all = events_all.drop_duplicates(['session_id', 'url', 'additional_url'])

In [22]:
events_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51092 entries, 0 to 42153
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   session_id         51092 non-null  int64  
 1   session_name       51092 non-null  object 
 2   url                51092 non-null  object 
 3   additional_url     5793 non-null   object 
 4   organization_id    35611 non-null  float64
 5   organization_name  51092 non-null  object 
 6   session_identity   51092 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 3.1+ MB


In [23]:
org_id_dict = dict(zip(organizations['Учреждение'], organizations['ID']))

In [24]:
from typing import List, Union, Optional

In [25]:
def get_org_id(org_name: str, org_dict: dict) -> Optional[int]:
    try:
        return org_dict[org_name]
    except KeyError:
        return None

In [26]:
events_all['organization_id'].fillna(events_all[events_all['organization_id'].isnull()]['organization_name'].apply(func=get_org_id, org_dict=org_id_dict), inplace=True)

In [27]:
events_all= events_all.astype({"organization_id": "int"})

In [28]:
events_1 = events_all[events_all['additional_url'].isnull()].drop('additional_url', axis=1)

In [29]:
events_2 = events_all[events_all['url'] == events_all['additional_url']].drop('additional_url', axis=1)

In [30]:
events_3 = events_all.loc[events_all['additional_url'].notna()].drop_duplicates(subset=['additional_url', 'url']).drop('additional_url', axis=1)

In [31]:
events_4 = events_all.loc[events_all['additional_url'].notna()].drop_duplicates(subset=['additional_url', 'url']).drop('url', axis=1).rename(columns = {'additional_url': 'url'})

In [32]:
event = pd.concat([events_1, events_2, events_3, events_4], ignore_index=True)

In [33]:
event.drop_duplicates(inplace=True)

In [34]:
event.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54608 entries, 0 to 56661
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   session_id         54608 non-null  int64 
 1   session_name       54608 non-null  object
 2   url                54608 non-null  object
 3   organization_id    54608 non-null  int32 
 4   organization_name  54608 non-null  object
 5   session_identity   54608 non-null  object
dtypes: int32(1), int64(1), object(4)
memory usage: 2.7+ MB


In [35]:
events_links = event['url'].unique()

In [36]:
len(events_links)

47736

In [37]:
click_links = click['url'].unique()

In [38]:
len(click_links)

19923

In [39]:
equal_links = []

for i in click_links:
    for j in events_links:
        if i == j:
            equal_links.append(i)

In [40]:
click_filter = click['url'].isin(equal_links)
click = click[click_filter]

In [41]:
diff = list(set(click_links) - set(equal_links))

In [42]:
len(diff)

166

In [43]:
cliks_add = pd.merge(click, event, how = 'left', on = 'url')

In [44]:
cliks_add.head()

,create_time,create_date,user_phone_details,buyer_mobile_phone,user_id,url,session_id,session_name,organization_id,organization_name,session_identity
0,2021-11-28 21:32:53,2021-11-28,"Android, 10 (SDK 29)",ce730a5e7bb6372aad126e934320b75c,08014dcd786881f349938c9255e9df36,https://theatreofnations.ru/performances/shved...,717865,Спектакль «Шведская спичка»,7577,ФГБУК «Государственный театр наций»,Спектакль
1,2021-11-28 21:41:52,2021-11-28,"Android, 10 (SDK 29)",ce730a5e7bb6372aad126e934320b75c,08014dcd786881f349938c9255e9df36,https://theatreofnations.ru/performances/shved...,717865,Спектакль «Шведская спичка»,7577,ФГБУК «Государственный театр наций»,Спектакль
2,2021-11-28 21:25:13,2021-11-28,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://tickets.hermitagemuseum.org/event/7E06...,1309020,Экспозиции главного музейного комплекса Эрмитажа,2217,ФГБУК «Государственный Эрмитаж»,Экспозиции
3,2021-11-28 22:11:33,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://tickets.kreml.ru/ru/#id=1&sid=763,1261967,Программа «Посещение архитектурного ансамбля С...,1848,Музеи Московского Кремля,Программа
4,2021-11-28 22:20:10,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://ramt.ru/plays/play-217/,1247322,Спектакль «Цветы для Элджернона»,4938,ФГБУК «Российский государственный академически...,Спектакль


In [45]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2547928 entries, 0 to 2547927
Data columns (total 11 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   create_time         object
 1   create_date         object
 2   user_phone_details  object
 3   buyer_mobile_phone  object
 4   user_id             object
 5   url                 object
 6   session_id          int64 
 7   session_name        object
 8   organization_id     int32 
 9   organization_name   object
 10  session_identity    object
dtypes: int32(1), int64(1), object(9)
memory usage: 223.6+ MB


In [46]:
organizations = organizations.rename(
    columns= {'ID': 'organization_id','Категория': 'org_category', 'Код': 'org_region_number'}) 

In [47]:
cliks_add = pd.merge(cliks_add, organizations[['organization_id', 'org_region_number', 'org_category']],
                     how = 'left', on = 'organization_id')

In [48]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2547928 entries, 0 to 2547927
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_time         object 
 1   create_date         object 
 2   user_phone_details  object 
 3   buyer_mobile_phone  object 
 4   user_id             object 
 5   url                 object 
 6   session_id          int64  
 7   session_name        object 
 8   organization_id     int32  
 9   organization_name   object 
 10  session_identity    object 
 11  org_region_number   float64
 12  org_category        object 
dtypes: float64(1), int32(1), int64(1), object(10)
memory usage: 262.4+ MB


In [49]:
cliks_add = cliks_add[cliks_add['org_category'].notna()]

In [50]:
cliks_add['type'] = cliks_add[['org_category', 'session_identity']].apply(lambda x: '_'.join(x), axis = 1)

In [51]:
cliks_add['type'].value_counts().head(60)

Театры_Спектакль                                1263774
Музеи и галереи_Выставка                         156876
Театры_Балет                                     144426
Театры_Опера                                     126990
Музеи и галереи_Экскурсия                         80083
Театры_Концерт                                    63196
Театры_Мюзикл                                     48324
Концертные площадки_Концерт                       40955
Музеи и галереи_Программа                         34801
Музеи и галереи_Экспозиции                        32177
Музеи и галереи_Мастер-класс                      25449
Театры_Программа                                  22010
Театры_Оперетта                                   21907
Музеи и галереи_Экспозиция                        21286
Музеи и галереи_Лекция                            19612
Дворцы культуры и клубы_Концерт                   14698
Театры_Музыкальная                                14053
Музеи и галереи_Обзорная                        

In [52]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2547633 entries, 0 to 2547927
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_time         object 
 1   create_date         object 
 2   user_phone_details  object 
 3   buyer_mobile_phone  object 
 4   user_id             object 
 5   url                 object 
 6   session_id          int64  
 7   session_name        object 
 8   organization_id     int32  
 9   organization_name   object 
 10  session_identity    object 
 11  org_region_number   float64
 12  org_category        object 
 13  type                object 
dtypes: float64(1), int32(1), int64(1), object(11)
memory usage: 281.8+ MB


In [53]:
users = pd.read_csv(path + 'users_full.csv', sep=';')

In [54]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205191 entries, 0 to 1205190
Data columns (total 13 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   account_create_date    1205191 non-null  object 
 1   user_id                1205191 non-null  object 
 2   user_birth             1205191 non-null  object 
 3   app_open_date          1205191 non-null  object 
 4   user_phone_details     1205191 non-null  object 
 5   buyer_mobile_phone     1205191 non-null  object 
 6   user_phone_details_id  1205191 non-null  object 
 7   type_phone             1205191 non-null  int64  
 8   region                 765268 non-null   object 
 9   age                    1205191 non-null  int64  
 10  user_region            765268 non-null   float64
 11  diff                   1205191 non-null  object 
 12  in_count               1205191 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 119.5+ MB


In [55]:
cliks_add = pd.merge(cliks_add, users[['user_id', 'user_region']],
                     how = 'left', on = 'user_id')

In [56]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2547633 entries, 0 to 2547632
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_time         object 
 1   create_date         object 
 2   user_phone_details  object 
 3   buyer_mobile_phone  object 
 4   user_id             object 
 5   url                 object 
 6   session_id          int64  
 7   session_name        object 
 8   organization_id     int32  
 9   organization_name   object 
 10  session_identity    object 
 11  org_region_number   float64
 12  org_category        object 
 13  type                object 
 14  user_region         float64
dtypes: float64(2), int32(1), int64(1), object(11)
memory usage: 301.3+ MB


In [57]:
cliks_add = cliks_add[cliks_add['user_region'].notna()]

In [58]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1824497 entries, 0 to 2547632
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_time         object 
 1   create_date         object 
 2   user_phone_details  object 
 3   buyer_mobile_phone  object 
 4   user_id             object 
 5   url                 object 
 6   session_id          int64  
 7   session_name        object 
 8   organization_id     int32  
 9   organization_name   object 
 10  session_identity    object 
 11  org_region_number   float64
 12  org_category        object 
 13  type                object 
 14  user_region         float64
dtypes: float64(2), int32(1), int64(1), object(11)
memory usage: 215.8+ MB


In [ ]:
#cliks_add.to_csv(path + 'cliks_add_3.csv', sep=';', index=False)